# Load and install package

In [ ]:
!pip install pandas

In [1]:
import pandas as pd
import json, os, csv

# Process and Analysis

In [18]:
df_comments = pd.read_json('data/initial-data/scraping/comments_by_classes.json')
df_feedback = pd.read_json('data/initial-data/scraping/feedback_by_classes.json')
manual_collecting = pd.read_csv('data/results/engineering-1.csv')

In [19]:
df_comments = df_comments.drop_duplicates(subset=['user_statement'])
df_comments

,user_statement,judul,rating,type,kategori,pusat_inovasi,status,tema
0,"Permisi, Bapak/Ibu. Mohon maaf sebelumnya,",Kelas Online Pendalaman Manajemen Pelayanan Pu...,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
1,perencanaan yang disusun dalam pemerintah daer...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
2,Perlunya penguatan internalisasi RB diseluruh ASN,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
3,alhamdulillah,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
4,apa dampak jika dynamic governance tidak diimp...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
5,Bagaimana menerapkan Reformasi Birokrasi yang ...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
6,Paradigma administrasi apa yang saat ini ada d...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
7,seberapa efektif birokrasi tematik dalam menye...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema
8,Apa dasar hukum dari materi dalam kelas pembel...,Kelas Online Memahami Reformasi Birokrasi di I...,,komentar/pertanyaan,Smartclass,Semua Pusat Inovasi,publish,Semua tema


In [20]:
df_feedback['user_statement'] = df_feedback['user_statement'].astype(str).str.strip()
df_feedback = df_feedback[~df_feedback['user_statement'].str.fullmatch(r'[-.,!? ]+')]
df_feedback = df_feedback.drop_duplicates(subset=['user_statement'])
df_feedback

,user_statement,rating,kategori,pusat_inovasi,status,tema,judul,type
1,mantap,4,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Konsep dasar Sistem Merit,feedback
3,"keren, semoga ada kelas yang lain dengan narsu...",5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Konsep dasar Sistem Merit,feedback
4,Kelasnya seru,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Manajemen Talenta,feedback
6,OK,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Manajemen Talenta,feedback
10,terimakasih.. penjelasan dalam video sagat mud...,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback
11,Mantap,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback
12,Terima kasih atas ilmunya..,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback
13,yes,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback
14,Mudah dimengerti,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback
16,baik,5,Smartclass,Semua Pusat Inovasi,publish,Semua tema,Kelas Online Memahami Reformasi Birokrasi di I...,feedback


In [21]:
df_comments.to_csv('data/results/scrapping-comments.csv', index=False)
df_feedback.to_csv('data/results/scrapping-feedback.csv', index=False)

In [23]:
new_df = pd.concat([df_comments, df_feedback], axis=0)
new_df.to_csv('data/results/combine-feedback-comments.csv', index=False)

In [24]:
def read_csv(filename):
    with open(filename, 'r', encoding='utf-16', errors='ignore') as f:
        reader = csv.reader(f)
        rows = list(reader)  

    header = rows[0]
    data = rows[1:]

    data_clean = [row for row in data if len(row) == len(header)]
    df = pd.DataFrame(data_clean, columns=header)
    return df

In [25]:
review_users = os.listdir('data/initial-data/review-playstore')
df = pd.DataFrame({})

for review in review_users:
    temp = read_csv(f'data/initial-data/review-playstore/{review}')
    df = pd.concat([df, temp], axis=0)

In [27]:
df_review = df[['Star Rating', 'Review Text']].copy()
df_review = df_review.rename(columns={'Star Rating': 'rating', 'Review Text': 'user_statement'})
df_review['user_statement'] = df_review['user_statement'].astype(str).str.strip()
df_review['user_statement'] = df_review['user_statement'].replace('', pd.NA)
df_review['type'] = 'User Review Playstore'
df_review = df_review.dropna(subset=['user_statement']).reset_index(drop=True)

df_review

,rating,user_statement,type
0,5,Siap. Semoga bermanfaat,User Review Playstore
1,5,good apps for learning good governance,User Review Playstore
2,5,"aplikasinya bagus sekali, sangat mudah penggun...",User Review Playstore
3,5,"Banyak fitur dan kelas menarik, aplikasinya sa...",User Review Playstore
4,5,"Keren aplikasinya, sangat bermanfaat",User Review Playstore
5,5,Mantap dan membantu,User Review Playstore
6,5,Aplikasi yang sabgat bermanfaat untuk ASN,User Review Playstore
7,5,"Aplikasinya sangat bermanfaat buat ASN, terima...",User Review Playstore
8,5,Jos,User Review Playstore
9,5,Aplikasi ini sangat membantu untuk mencari kel...,User Review Playstore


In [29]:
df_join = pd.concat([df_review, new_df], axis=0)
df_join = df_join.drop(['judul', 'kategori', 'pusat_inovasi', 'status', 'tema'], axis=1)
df_join

,rating,user_statement,type
0,5,Siap. Semoga bermanfaat,User Review Playstore
1,5,good apps for learning good governance,User Review Playstore
2,5,"aplikasinya bagus sekali, sangat mudah penggun...",User Review Playstore
3,5,"Banyak fitur dan kelas menarik, aplikasinya sa...",User Review Playstore
4,5,"Keren aplikasinya, sangat bermanfaat",User Review Playstore
5,5,Mantap dan membantu,User Review Playstore
6,5,Aplikasi yang sabgat bermanfaat untuk ASN,User Review Playstore
7,5,"Aplikasinya sangat bermanfaat buat ASN, terima...",User Review Playstore
8,5,Jos,User Review Playstore
9,5,Aplikasi ini sangat membantu untuk mencari kel...,User Review Playstore


In [30]:
df_join.to_csv('data/results/combine-base.csv', index=False)